In [2]:
import requests
import csv
import pandas as pd
import re

from datetime import datetime
from tqdm import tqdm
import time
import json
import math

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [84]:
# pip install folium

### Получение данных через API

#### Формирование запроса API 2GIS
[API 2GIS](https://docs.2gis.com/ru/api/search/places/overview)

In [180]:
API_KEY = "API_KEY"

url = 'https://catalog.api.2gis.com/3.0/items'
params = {
    'key': API_KEY,
    'q': 'рестораны',
    'location': '30.373517,59.939079',  # Точка между офисами Дегтярный пер., д. 11Б и пер. Виленский, дом 14 литера А '
    'radius': 2500,  # метры
    'type': 'branch',  # branch — компания
    'fields':
    'items.point,items.address,items.adm_div,items.full_address_name,items.geometry.centroid,items.rubrics,items.org,items.contact_groups,items.schedule,items.access_comment,items.access,items.capacity,items.description,items.external_content,items.flags,items.floors,items.floor_plans,items.is_paid,items.for_trucks,items.paving_type,items.is_incentive,items.purpose,items.level_count,items.links,items.links.database_entrances.apartments_info,items.name_ex,items.reviews,items.statistics,context_rubrics,dym,filters,hash,items.ads.options,items.attribute_groups,items.context,items.dates.deleted_at,items.dates.updated_at,items.dates,items.geometry.style,items.group,items.metarubrics,items.delivery,items.has_goods,items.has_pinned_goods,items.has_realty,items.has_audiogid,items.has_discount,items.has_exchange,items.is_main_in_group,items.city_alias,items.detailed_subtype,items.alias,items.caption,items.is_promoted,items.routes,items.directions,items.barrier,items.is_routing_available,items.entrance_display_name,items.locale,items.reg_bc_url,items.region_id,items.segment_id,items.stat,items.stop_factors,items.has_apartments_info,items.timezone,items.timezone_offset,items.comment,items.station_id,items.platforms,items.sources,items.structure_info.material,items.structure_info.apartments_count,items.structure_info.porch_count,items.structure_info.floor_type,items.structure_info.gas_type,items.structure_info.year_of_construction,items.structure_info.elevators_count,items.structure_info.is_in_emergency_state,items.structure_info.project_type,items.structure_info.chs_name,items.structure_info.chs_category,items.route_logo,items.order_with_cart,items.is_deleted,items.search_attributes,items.congestion,items.poi_category,items.has_dynamic_congestion,items.temporary_unavailable_atm_services,items.marker_alt,items.floor_id,items.purpose_code,request_type,search_attributes,widgets,items.name_back,items.value_back,items.ev_charging_station,items.ski_lift,items.has_ads_model',
    'page_size': 10,
    'page': 1,
    'sort': 'distance'
}

In [181]:
def build_url(url, params):
    query_string = ''

    for key, value in params.items():
      query_string += f"{key}={value}&"

    if query_string.endswith('&'):
        query_string = query_string[:-1]

    full_url = f"{url}?{query_string}"

    return full_url

In [182]:
url = build_url(url, params)
url

'https://catalog.api.2gis.com/3.0/items?key=API_KEY&q=рестораны&location=30.373517,59.939079&radius=2500&type=branch&fields=items.point,items.address,items.adm_div,items.full_address_name,items.geometry.centroid,items.rubrics,items.org,items.contact_groups,items.schedule,items.access_comment,items.access,items.capacity,items.description,items.external_content,items.flags,items.floors,items.floor_plans,items.is_paid,items.for_trucks,items.paving_type,items.is_incentive,items.purpose,items.level_count,items.links,items.links.database_entrances.apartments_info,items.name_ex,items.reviews,items.statistics,context_rubrics,dym,filters,hash,items.ads.options,items.attribute_groups,items.context,items.dates.deleted_at,items.dates.updated_at,items.dates,items.geometry.style,items.group,items.metarubrics,items.delivery,items.has_goods,items.has_pinned_goods,items.has_realty,items.has_audiogid,items.has_discount,items.has_exchange,items.is_main_in_group,items.city_alias,items.detailed_subtype,ite

In [81]:
# Пример отправки запроса
# response = requests.get(url)

# data = response.json()
# items = data.get('result', {}).get('items', [])

#### Отправка запросов к API 2GIS (получение всех данных)

In [82]:
def flatten_json(nested_json, parent_key='', sep='_'):
    """
    Рекурсивно разворачивает вложенный JSON в одномерный словарь.
    
    Параметры:
    nested_json (dict): Вложенный JSON для разворачивания.
    parent_key (str): Ключ родителя, который добавляется к ключам на каждом уровне (по умолчанию пустая строка).
    sep (str): Разделитель, используемый для объединения ключей (по умолчанию '_').
    
    Возвращает:
    dict: Новый словарь с "плоскими" ключами.
    """
    items = []
    for k, v in nested_json.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_json(v, new_key, sep=sep).items())
        elif isinstance(v, list):
            if v and isinstance(v[0], dict):
                for i, sub_dict in enumerate(v):
                    items.extend(flatten_json(sub_dict, f"{new_key}_{i}", sep=sep).items())
            else:
                items.append((new_key, ", ".join(map(str, v))))
        else:
            items.append((new_key, v))
    return dict(items)

In [83]:
def get_all_data(url, params, max_pages=5, start_page=1):
    """
    Получает все данные с API, обрабатывает страницы с результатами, начиная с start_page, и сохраняет их в одном списке.
    
    Параметры:
    url (str): Базовый URL для API-запроса.
    params (dict): Параметры запроса, включая параметры для пагинации.
    max_pages (int): Максимальное количество страниц для получения (по умолчанию 5).
    start_page (int): Страница, с которой начинается получение данных (по умолчанию 1).

    Возвращает:
    list: Список всех данных, собранных с нескольких страниц.
    """
    all_flat_data = []
    page = start_page

    with tqdm(total=max_pages, desc="Загрузка данных", ncols=100) as pbar:
        while True:
            # time.sleep(1)

            if page > max_pages:
                # print("Достигнут предел по количеству страниц.")
                break

            params['page'] = page
            request_url = build_url(url, params)

            pbar.set_postfix(page=page)
            pbar.update(1)

            response = requests.get(request_url)
            data = response.json()

            if 'result' in data and 'items' in data['result']:
                for item in data['result']['items']:
                    flat_data = flatten_json(item)
                    all_flat_data.append(flat_data)

                if len(data['result']['items']) < params['page_size']:
                    print(f"Данные на странице {page} меньше заявленного размера. Останавливаемся.")
                    break
                else:
                    page += 1
            else:
                print("Ошибка при получении данных")
                break

    return all_flat_data

In [311]:
max_pages = 150
params['location'] = '30.373517,59.939079'
url = 'https://catalog.api.2gis.com/3.0/items'
all_flat_data = get_all_data(url, params, max_pages=max_pages, start_page = 1)

Загрузка данных:   4%|█▌                                    | 6/150 [00:11<04:43,  1.97s/it, page=6]

Ошибка при получении данных


Изначально была идея получить информацию о всех кафе/ресторанах в некотором радиусе от офисов/центра Санкт-Петербурга, но у API 2GIS есть ограничение на количество ресторанов в одном ответе (до 10) и количество страниц (до 5). 

In [69]:
df = pd.DataFrame(all_flat_data)

In [70]:
print (len(df))
df = df.drop_duplicates(subset='id', keep='first')
print (len(df))

30
30


In [71]:
df[['address_name',
    'name_ex_primary', 'name_ex_extension',
    'reviews_general_rating','reviews_general_review_count', 'reviews_general_review_count_with_stars',
    "links_entrances_0_geometry_points"]].head(3)

,address_name,name_ex_primary,name_ex_extension,reviews_general_rating,reviews_general_review_count,reviews_general_review_count_with_stars,links_entrances_0_geometry_points
0,"9-я Советская улица, 3",Harbor,ресторан,4.6,7.0,9.0,POINT(30.370701625633792 59.938567148391527)
1,"Суворовский проспект, 27",Kroo cafe,французский ресторан,4.3,104.0,160.0,POINT(30.374200653496967 59.937567679721774)
2,"Греческий проспект, 29",Osteria Betulla,остерия,4.3,167.0,285.0,POINT(30.370271781770345 59.938672444011928)


In [15]:
df.to_csv('data/data_restaurants.csv', index=False)

Поэтому буду использовать другой подход:

Я начинаю с того, что беру небольшой радиус (например, 700 метров) от исходной точки (одного из офисов) и запрашиваю через API информацию о ресторанах в данной области. Получаю данные о 5 страницах, на каждой из которых по 10 ресторанов. После этого я сдвигаюсь на некоторых шаг от исходной точки в разные направления: на север, юг, запад, восток, северо-запад и т. д. и делаю запросы в этих новых местах. Таким образом, я постепенно охватываю более широкую территорию.

#### Отправка запросов к API 2GIS (получение данных из маленьких окружностей)

In [7]:
def get_shifted_coordinates(lat, lon, shift_count_1, shift_count_2, distance_step):
    """
    Функция для получения координат после сдвигов в различных направлениях.

    :param lat: начальная широта
    :param lon: начальная долгота
    :param shift_count_1: количество сдвигов в каждом направлении на север/юг/...
    :param shift_count_2: количество сдвигов в каждом направлении на северо-запад/юг-восток/...
    :distance_step: шаг сдвига в метрах
    :return: список координат (широта, долгота) для каждого сдвига
    """

    lat_degree = 111320

    lon_degree = 111320 * math.cos(math.radians(lat))

    delta_lat = distance_step / lat_degree
    delta_lon = distance_step / lon_degree

    directions = [
        ('north', delta_lat, 0),  # Север
        ('south', -delta_lat, 0),  # Юг
        ('east', 0, delta_lon),    # Восток
        ('west', 0, -delta_lon),   # Запад
        ('north-east', delta_lat, delta_lon),  # Северо-восток
        ('north-west', delta_lat, -delta_lon), # Северо-запад
        ('south-east', -delta_lat, delta_lon), # Юго-восток
        ('south-west', -delta_lat, -delta_lon) # Юго-запад
    ]

    shifted_coordinates = [('start', lat, lon)]

    for direction, d_lat, d_lon in directions[:4]:
        for i in range(1, shift_count_1 + 1):
            new_lat = lat + i * d_lat
            new_lon = lon + i * d_lon
            shifted_coordinates.append((direction, new_lat, new_lon))

    for direction, d_lat, d_lon in directions[4:]:
        for i in range(1, shift_count_2 + 1):
            new_lat = lat + i * d_lat
            new_lon = lon + i * d_lon
            shifted_coordinates.append((direction, new_lat, new_lon))

    return shifted_coordinates

##### Визуализация

In [10]:
import folium

coord_off_12 = '30.373517,59.939079'  # Точка между офисами Дегтярный пер., д. 11Б и пер. Виленский, дом 14 литера А '
lat = float(coord_off_12.split(",")[1])
lon = float(coord_off_12.split(",")[0])
shift_count_1 = 2
shift_count_2 = 1
distance_step = 900
radius = 700

coordinates = get_shifted_coordinates(lat, lon, shift_count_1, shift_count_2, distance_step)

m = folium.Map(location=[lat, lon], zoom_start=14)

folium.Marker([lat, lon], popup="Стартовая точка").add_to(m)

for direction, d_lat, d_lon in coordinates:
      folium.Circle(
          location=[d_lat, d_lon],
          radius=radius,
          color='blue' if direction in ['north', 'south', 'east', 'west'] else 'green',
          fill=True,
          fill_color='blue' if direction in ['north', 'south', 'east', 'west'] else 'green',
          fill_opacity=0.4
      ).add_to(m)

# m.save("map_with_multiple_circles.html")
m

Визуализация данного подхода

#### Получение данных через API

In [183]:
API_KEY = "API_KEY"

url = 'https://catalog.api.2gis.com/3.0/items'
params = {
    'key': API_KEY,
    'q': 'рестораны',
    'location': '30.373517,59.939079',  # Точка между офисами Дегтярный пер., д. 11Б и пер. Виленский, дом 14 литера А '
    'radius': 700,  # метры
    'type': 'branch',  # branch — компания
    'fields':
    'items.point,items.address,items.adm_div,items.full_address_name,items.geometry.centroid,items.rubrics,items.org,items.contact_groups,items.schedule,items.access_comment,items.access,items.capacity,items.description,items.external_content,items.flags,items.floors,items.floor_plans,items.is_paid,items.for_trucks,items.paving_type,items.is_incentive,items.purpose,items.level_count,items.links,items.links.database_entrances.apartments_info,items.name_ex,items.reviews,items.statistics,context_rubrics,dym,filters,hash,items.ads.options,items.attribute_groups,items.context,items.dates.deleted_at,items.dates.updated_at,items.dates,items.geometry.style,items.group,items.metarubrics,items.delivery,items.has_goods,items.has_pinned_goods,items.has_realty,items.has_audiogid,items.has_discount,items.has_exchange,items.is_main_in_group,items.city_alias,items.detailed_subtype,items.alias,items.caption,items.is_promoted,items.routes,items.directions,items.barrier,items.is_routing_available,items.entrance_display_name,items.locale,items.reg_bc_url,items.region_id,items.segment_id,items.stat,items.stop_factors,items.has_apartments_info,items.timezone,items.timezone_offset,items.comment,items.station_id,items.platforms,items.sources,items.structure_info.material,items.structure_info.apartments_count,items.structure_info.porch_count,items.structure_info.floor_type,items.structure_info.gas_type,items.structure_info.year_of_construction,items.structure_info.elevators_count,items.structure_info.is_in_emergency_state,items.structure_info.project_type,items.structure_info.chs_name,items.structure_info.chs_category,items.route_logo,items.order_with_cart,items.is_deleted,items.search_attributes,items.congestion,items.poi_category,items.has_dynamic_congestion,items.temporary_unavailable_atm_services,items.marker_alt,items.floor_id,items.purpose_code,request_type,search_attributes,widgets,items.name_back,items.value_back,items.ev_charging_station,items.ski_lift,items.has_ads_model',
    'page_size': 10,
    'page': 1,
    'sort': 'distance'
}

In [72]:
coord_off = ['30.373517,59.939079', '30.323836,59.901766'] # Первая точка между офисами Дегтярный пер., д. 11Б и пер. Виленский, дом 14, а вторая офис на Киевская улица, 5к4
numbers_off = [[1, 2], [3]]

shift_count_1 = 2
shift_count_2 = 1
distance_step = 900
radius = 700
params['radius'] = radius
max_pages = 5
all_flat_data = []

for name in ['рестораны', 'кафе']:
    params['q'] = name
    for i, coord in enumerate(coord_off):
        print (f"\nОбработка {i+1} точки ({name})")
        lon, lat = map(float, coord.split(','))
    
        coordinates = get_shifted_coordinates(lat, lon, shift_count_1, shift_count_2, distance_step)
        for direction, d_lat, d_lon in coordinates:
            params['location'] = f'{d_lon},{d_lat}'
            res = get_all_data(url, params, max_pages=max_pages, start_page = 1)
            for rest in res:
                for j in numbers_off[i]:  
                    rest[f"near_office_{j}"] = True
            all_flat_data.extend(res)


Обработка 1 точки (рестораны)


Загрузка данных: 100%|████████████████████████████████████████| 5/5 [00:01<00:00,  3.62it/s, page=5]



Обработка 2 точки (рестораны)


Загрузка данных: 100%|████████████████████████████████████████| 5/5 [00:01<00:00,  3.71it/s, page=5]



Обработка 1 точки (кафе)


Загрузка данных: 100%|████████████████████████████████████████| 5/5 [00:01<00:00,  2.78it/s, page=5]



Обработка 2 точки (кафе)


Загрузка данных: 100%|████████████████████████████████████████| 5/5 [00:01<00:00,  2.71it/s, page=5]


Запрашиваем данные не только о ресторанах, но и о кафе

In [73]:
df = pd.DataFrame(all_flat_data)

In [74]:
df.head()

,address_building_id,address_components_0_number,address_components_0_street,address_components_0_street_id,address_components_0_type,address_postcode,address_comment,address_name,adm_div_0_id,adm_div_0_name,...,links_nearest_parking_85_id,links_nearest_parking_86_id,links_nearest_parking_87_id,links_nearest_parking_88_id,links_nearest_parking_89_id,links_nearest_parking_90_id,links_nearest_parking_91_id,attribute_groups_7_attributes_5_id,attribute_groups_7_attributes_5_name,attribute_groups_7_attributes_5_tag
0,5348660212708350,3,9-я Советская улица,5348763291879950,street_number,191036,1 этаж,"9-я Советская улица, 3",1,Россия,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5348660212708114,27,Суворовский проспект,5348763291879693,street_number,191036,1 этаж,"Суворовский проспект, 27",1,Россия,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5348660212708348,29,Греческий проспект,5348763291879834,street_number,191036,цокольный этаж,"Греческий проспект, 29",1,Россия,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5348660212794938,4а,8-я Советская улица,5348763291879943,street_number,191036,2 этаж,"8-я Советская улица, 4а",1,Россия,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5348660212708111,25,Суворовский проспект,5348763291879693,street_number,191036,1 этаж,"Суворовский проспект, 25",1,Россия,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
df.to_csv('data/data_restaurants1.csv', index=False)

In [76]:
print (len(df))
df = df.drop_duplicates(subset='id', keep='first')
print (len(df))

2600
1029


Области пересекаются, поэтому мы выше удалили все дубликаты 

In [77]:
df.to_csv('data/data_restaurants2.csv', index=False)

### Получение данных от парсера

Есть парсер, который собирает собирает информацию по ссылке 2GIS. С его помощью было получено более 1000 ресторанов Санкт-Петербурга, и эти данные обрабатываются и сохраняются для дальнейшего использования. Например, если человек решит пообедать не в одном из трех основных офисов, а в другом месте, то данные будут под рукой.

Кроме того, рестораны, расположенные рядом с основными офисами, будут помечены отдельно. Это позволит значительно упростить поиск ближайшего ресторана к офису, потому что не придется искать среди всей базы.

In [144]:
with open("data/data_from_parser.json", "r", encoding="utf-8-sig") as file:
    data_from_parser = json.load(file)

In [145]:
all_flat_data_from_parser = []
for item in data_from_parser:
    flat_data = flatten_json(item)
    all_flat_data_from_parser.append(flat_data)
df_from_parser = pd.DataFrame(all_flat_data_from_parser)

In [146]:
print (len(df_from_parser))
df_from_parser = df_from_parser.drop_duplicates(subset='id', keep='first')
print (len(df_from_parser))

1317
1317


In [147]:
df_from_parser['id'] = df_from_parser['id'].apply(lambda x: x.split('_')[0])

In [148]:
df_from_parser[['address_name',
    'name_ex_primary', 'name_ex_extension',
    'reviews_general_rating','reviews_general_review_count', 'reviews_general_review_count_with_stars',
    "links_entrances_0_geometry_points"]].head(3)

,address_name,name_ex_primary,name_ex_extension,reviews_general_rating,reviews_general_review_count,reviews_general_review_count_with_stars,links_entrances_0_geometry_points
0,"улица Льва Толстого, 9",Паруса на крыше,панорамный ресторан,4.6,2501.0,2909.0,POINT(30.315631395287664 59.965326500630383)
1,"Невский проспект, 20",Большой Грузинский,ресторан,4.7,1205.0,6145.0,POINT(30.321680111421749 59.93613317547684)
2,"2-я Советская улица, 12",Caribia,NaN,4.9,164.0,225.0,POINT(30.369720216185897 59.931498818572834)


In [149]:
df_from_parser.to_csv('data/data_restaurants_from_parser.csv', index=False)

### Обработка полученных данных

#### Удаляем неинформативные столбцы


In [151]:
df = pd.read_csv('data/data_restaurants2.csv');
df_from_parser = pd.read_csv('data/data_restaurants_from_parser.csv');

In [154]:
print (len(df_from_parser))
df_from_parser = df_from_parser[~df_from_parser['id'].isin(df['id'])]
print (len(df_from_parser))

1317
804


In [156]:
df_cleaned = pd.concat([df, df_from_parser], ignore_index=True)

In [159]:
# df_cleaned = df
df_cleaned = df_cleaned.loc[:, ~df_cleaned.columns.str.startswith('links_nearest_')] # убираем информацию о ближайших стациях метро и парковках
df_cleaned = df_cleaned.loc[:, ~ df_cleaned.columns.str.contains('attributes_\d+_id')] # убираем id атрибутов (тегов)
df_cleaned = df_cleaned.loc[:, ~ df_cleaned.columns.str.contains('attributes_\d+_tag')] # убираем id атрибутов (тегов)

df_cleaned = df_cleaned.loc[:, ~ df_cleaned.columns.str.contains('attribute_groups_\d+_icon_url')] # убираем url картонок для групп информации (меню, напитки и тд)

#убираем информацию о том, что данная группа атрибутов является основной для данного объекта или типа объектов
df_cleaned = df_cleaned.loc[:, ~ df_cleaned.columns.str.contains('_is_primary')]

#убираем название тегов
df_cleaned = df_cleaned.loc[:, ~ df_cleaned.columns.str.contains('_tag')]

#убираем список идентификаторов рубрик, к которым относится эта группа атрибутов
df_cleaned = df_cleaned.loc[:, ~ df_cleaned.columns.str.contains('_rubric_ids')]


#убираем лишнюю информацию о стране и городе
df_cleaned.drop([ 'address_components_0_street_id',
 'address_components_0_type',
 'address_postcode',
 'adm_div_0_id',
 'adm_div_0_name',
 'adm_div_0_type',
 'adm_div_1_id',
 'adm_div_1_type',
 'adm_div_2_city_alias',
 'adm_div_2_flags_is_default',
 'adm_div_2_flags_is_region_center',
 'adm_div_2_id',
 'adm_div_2_is_default',
 'adm_div_2_type',
 'adm_div_3_id',
 'adm_div_3_type',
 'adm_div_4_detailed_subtype',
 'adm_div_4_id',
 'adm_div_4_type',
 'alias'], axis=1, inplace=True, errors='ignore')

df_cleaned = df_cleaned.loc[:, ~ df_cleaned.columns.str.contains('_type')]

#убираем булевые неинформативные данные
df_cleaned = df_cleaned.loc[:, ~ df_cleaned.columns.str.contains('_is_context')]
df_cleaned.drop([ 'has_ads_model',
 'has_apartments_info',
 'has_audiogid',
 'has_discount',
 'has_dynamic_congestion',
 'has_exchange',
 'has_goods',
 'has_pinned_goods',
 'has_realty',
 'is_promoted',
 'is_routing_available'], axis=1, inplace=True, errors='ignore')


df_cleaned.drop([ 'marker_alt', 'name_ex_short_name', 'org_branch_count', 'org_name', 'poi_category', 'region_id',
 'reviews_items_0_is_reviewable',
 'reviews_items_1_is_reviewable',
 'reviews_items_1_rating',
 'reviews_items_1_review_count',
 'reviews_org_rating',
 'reviews_org_review_count',
 'reviews_org_review_count_with_stars',
 'reviews_rating',
 'reviews_review_count',
 'rubrics_0_alias',
 'rubrics_0_id',
 'rubrics_0_kind',
 'rubrics_0_name',
 'rubrics_0_parent_id',
 'rubrics_0_short_id', 'caption',
 'links_entrances_0_is_visible_on_map', 'locale', 'reg_bc_url',
 'segment_id',	'stat_adst',	'stat_is_advertised',	'stat_rubr',	'timezone',	'timezone_offset',	'type', 'address_building_name',
 'dates_created_at','dates_updated_at','flags_photos', 'org_id', 'city_alias'], axis=1, inplace=True, errors='ignore')

df_cleaned = df_cleaned.loc[:, ~ df_cleaned.columns.str.contains('external_content')]


df_cleaned = df_cleaned.loc[:, ~ df_cleaned.columns.str.contains('rubrics_\d+_alias')]
df_cleaned = df_cleaned.loc[:, ~ df_cleaned.columns.str.contains('rubrics_\d+_kind')]
df_cleaned = df_cleaned.loc[:, ~ df_cleaned.columns.str.contains('rubrics_\d+_id')]
df_cleaned = df_cleaned.loc[:, ~ df_cleaned.columns.str.contains('rubrics_\d+_parent_id')]
df_cleaned = df_cleaned.loc[:, ~ df_cleaned.columns.str.contains('rubrics_\d+_short_id')]
df_cleaned = df_cleaned.loc[:, ~ df_cleaned.columns.str.contains('is_reviewable')]
df_cleaned = df_cleaned.loc[:, ~ df_cleaned.columns.str.contains('is_award')]
df_cleaned = df_cleaned.loc[:, ~ df_cleaned.columns.str.contains('_icon_url')]
df_cleaned = df_cleaned.loc[:, ~ df_cleaned.columns.str.contains('_recommendation_count')]
df_cleaned = df_cleaned.loc[:, ~ df_cleaned.columns.str.contains('flags_badges_')]
df_cleaned = df_cleaned.loc[:, ~ df_cleaned.columns.str.contains('flags_badges_')]



# Удаляем лишнюю информацию о месте
df_cleaned = df_cleaned.loc[:, ~ df_cleaned.columns.str.contains('_geometry_normals')]
df_cleaned = df_cleaned.loc[:, ~ df_cleaned.columns.str.contains('_geometry_vectors')]
df_cleaned = df_cleaned.loc[:, ~ df_cleaned.columns.str.contains('links_entrances_\d+_id')]
# df_cleaned.drop([ 'point_lat','point_lon'], axis=1, inplace=True, errors='ignore')

In [160]:
df_cleaned.to_csv('data/data_restaurants_cleaned.csv', index=False)

#### Модель для классификации блюд

In [161]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification

model_name = "DeepPavlov/rubert-base-cased"
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
model.load_state_dict(torch.load('models/dish_classifier/dish_classifier_model.pth'))
model.eval()

tokenizer = BertTokenizer.from_pretrained('models/dish_classifier/dish_tokenizer')

device = "cpu" #torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device);

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [162]:
def tokenize_function(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=128)

def classify_dishes_for_row(row, model, tokenizer):
    dishes = set()

    texts = []
    for col, val in row.items():
        texts.append(str(val))

    if not texts:
        return dishes

    inputs = tokenize_function(texts)

    dataset = TensorDataset(
        torch.tensor(inputs['input_ids']),
        torch.tensor(inputs['attention_mask'])
    )
    dataloader = DataLoader(dataset, batch_size=20)

    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask = [x.to(device) for x in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            predictions = torch.argmax(logits, dim=-1).cpu().numpy()

            for prediction, text in zip(predictions, texts):
                if prediction == 1:  # 1 - это блюдо
                    dishes.add(text)

    return dishes

#### Разбиваем столбцы на группы по тематике


In [ ]:
df_processed = pd.DataFrame()

# Обработка времени работы (создание названия колонок)
working_hours_col = []
for day in ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']:
    working_hours_col.append(f'schedule_{day}_working_hours_0_from')
    working_hours_col.append(f'schedule_{day}_working_hours_0_to')



# Обрабатываем данные по условиям
for index, row in df_cleaned.iterrows():
    row_copy = row.copy()
    row_copy.drop(working_hours_col, inplace=True)
    
    near_office_columns = [col for col in df_cleaned.columns if col.startswith('near_office')]
    main_info_col = ['name', 'reviews_general_rating', # Основная инфа 'full_address_name',
                     'id', 'name_ex_extension', 'name_ex_primary', # Подробнее про название
                     'address_building_id', 'address_components_0_number','address_components_0_street', # Подробнее про расположение
                     'address_name', 'adm_div_1_name', 'adm_div_2_name', 'adm_div_3_name', 'adm_div_4_name', # Подробнее про расположение
                     'links_entrances_0_geometry_points', 'point_lat','point_lon', # Координата входа в заведение
                     'reviews_general_review_count', 'reviews_general_review_count_with_stars' # Подробнее про рейтинг
                     ]
    main_info_col.extend(near_office_columns)
    
    average_bill_col = []
    payment_methods_col = []
    cuisines_col = []
    section_titles_col = []

    average_bills_text = []
    average_bill = set()
    payment_methods = set()
    cuisines = set()


    # Обработка основной информации
    df_processed.loc[index, main_info_col] = row_copy[main_info_col].values.tolist()
    row_copy.drop(main_info_col, inplace=True, errors='ignore')
    row_copy = row_copy.dropna(how='all')

    # Обработка колонок c названиями категорий и разных услуг
    section_titles = ["Меню", "Напитки", "Способы оплаты", 'Ресторан / Кафе', 'Ресторан', 'Кафе', 'Услуги', 'Доставка', 'Дресс-код', 'Дополнительно', 'Можно с собакой', 'Пандус', 'Танцпол', 'Поминальные обеды', 'Настольные игры', 'Проведение банкетов']
    for i, val in row_copy.items():
        if any(word in str(val) for word in section_titles):
          section_titles_col.append(i)
    row_copy.drop(section_titles_col, inplace=True)


    # Обработка колонки со Средним чеком
    for i, val in row_copy.items():
        if 'Средний чек' in str(val):
          average_bill_col.append(i)
          average_bills_text.append(val)
    for val in average_bills_text:
        number = re.search(r'\d+', str(val))
        if number:
            average_bill.add(int(number.group(0)))
    row_copy.drop(average_bill_col, inplace=True)

    # Обработка Способов оплаты
    payment_methods_all = ["Наличный расчёт", "Оплата по QR-коду",
                   "Оплата картой", "Оплата через банк",
                   "Перевод с карты"]
    for i, val in row_copy.items():
        if any(word in str(val) for word in payment_methods_all):
          payment_methods_col.append(i)
          payment_methods.add(val)
    row_copy.drop(payment_methods_col, inplace=True)


    # Обработка Разных видов кухни
    for i, val in row_copy.items():
        if 'кухня' in str(val).lower() and len(str(val))<70:
          cuisines_col.append(i)
          if re.match(r'^[a-zа-яё]+ кухня$', str(val).lower()):
            cuisines.add(val)
    row_copy.drop(cuisines_col, inplace=True)

    # Записываем уникальные значения в итоговые колонки
    if average_bill:
        df_processed.at[index, 'Average bill'] = max(average_bill)
    if payment_methods:
        df_processed.at[index, 'Payment'] = '; '.join(payment_methods)
    if cuisines:
        df_processed.at[index, 'Cuisine'] = '; '.join(cuisines)


    # Обработка дополнительных признаков (Чай с собой; Кофе с собой; Заказ навынос; Заказ столиков)
    add_params_col = []
    for i, val in row_copy.items():
        if 'чай с собой' in str(val).lower() or 'кофе с собой' in str(val).lower():
          df_processed.at[index, 'has_tea_coffee_takeaway'] = True
          add_params_col.append(i)
        elif 'заказ навынос' in str(val).lower() :
          df_processed.at[index, 'has_takeaway'] = True
          add_params_col.append(i)
        elif 'заказ столиков' in str(val).lower() :
          df_processed.at[index, 'has_reservations'] = True
          add_params_col.append(i)
        elif 'wi-fi' in str(val).lower() :
          df_processed.at[index, 'has_wifi'] = True
          add_params_col.append(i)
        elif 'живая музыка' in str(val).lower() :
          df_processed.at[index, 'has_live_music'] = True
          add_params_col.append(i)
        elif 'vip-зал' in str(val).lower() :
          df_processed.at[index, 'has_vip_lounge'] = True
          add_params_col.append(i)
        elif 'летняя терраса' in str(val).lower() :
          df_processed.at[index, 'has_summer_terrace'] = True
          add_params_col.append(i)
        elif 'банкетные залы' in str(val).lower() :
          df_processed.at[index, 'has_banquet_halls'] = True
          add_params_col.append(i)
            
        elif 'ланч' in str(val).lower() :
          df_processed.at[index, 'has_lunch'] = True
          add_params_col.append(i)
          if '₽' in str(val).lower():
            number = re.search(r'\d+', str(val))
            if number:
                df_processed.at[index, 'lunch_price_start'] = int(number.group(0))
        elif 'завтрак' in str(val).lower() :
            df_processed.at[index, 'has_breakfast'] = True
            add_params_col.append(i)
            breakfast_pattern = r'Завтрак (\d{2}:\d{2})-(\d{2}:\d{2})'
    
            res = re.search(breakfast_pattern, str(val).lower())
            if res:
                df_processed.at[index, 'breakfast_time_start'] = int(res.group(1))
                df_processed.at[index, 'breakfast_time_end'] = int(res.group(2))
        
        elif 'этаж' in str(val).lower() :
            add_params_col.append(i)
            pattern = r"(\d+)(?:\s*-\s*(\d+))?\s*этаж(?:ей)?"
            res = re.search(pattern, str(val).lower())
            if res:
                if res.group(2):
                    df_processed.at[index, 'floor1'] = int(res.group(1))
                    df_processed.at[index, 'floor2'] = int(res.group(2))
                else:
                    df_processed.at[index, 'floor1'] = int(res.group(1))
                    df_processed.at[index, 'floor2'] = int(res.group(1))
        
        elif re.search(r'\d+', str(val)):
          add_params_col.append(i)
        elif 'ноутбук' in str(val).lower() :
          add_params_col.append(i)
        elif 'мест' in str(val).lower() :
          add_params_col.append(i)
        elif 'человек' in str(val).lower() :
          add_params_col.append(i)
        elif 'помещение' in str(val).lower() :
          add_params_col.append(i)
        elif 'премия' in str(val).lower() :
          add_params_col.append(i)
        elif 'лучшее' in str(val).lower() :
          add_params_col.append(i)
        elif 'лучший' in str(val).lower() :
          add_params_col.append(i)
        elif 'false' in str(val).lower() :
          add_params_col.append(i)
        elif 'true' in str(val).lower() :
          add_params_col.append(i)

    row_copy.drop(add_params_col, inplace=True)

    dishes = classify_dishes_for_row(row_copy, model, tokenizer)
    if dishes:
        df_processed.at[index, 'Dishes'] = '; '.join(dishes)


def convert_to_datetime(time_str):
    if pd.isna(time_str):
        return None
    if time_str == '24:00':
        time_str = '00:00'
    return datetime.strptime(time_str, '%H:%M').time()

for day in ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']:
    from_col = f'schedule_{day}_working_hours_0_from'
    to_col = f'schedule_{day}_working_hours_0_to'
    df_processed[f'{day}_from'] = df_cleaned[from_col].apply(convert_to_datetime)
    df_processed[f'{day}_to'] = df_cleaned[to_col].apply(convert_to_datetime)

In [167]:
(df_processed["Dishes"].values)[:3]

array(['Детское меню; Стейк-хаус; Салаты; Картофель-фри; Куриные крылышки; Десерты; Шведский стол; Супы; Мидии; Паста; Горячие вторые блюда; Бургеры; Стейки; Пельмени',
       'Салаты; Десерты; Кондитерская; Глинтвейн; Супы; Горячие вторые блюда; Паста',
       'Паста; Супы; Пицца; Римская пицца'], dtype=object)

In [168]:
df_processed['id'] = df_processed['id'].astype(int)
df_processed['address_building_id'] = df_processed['address_building_id'].astype(int)

In [169]:
df_processed.to_csv('data/data_restaurants_processed.csv', index=False)

#### Добавляем подсчет расстояния до каждого из трех офисов

In [170]:
import math

def haversine(lat1, lon1, lat2, lon2):
    R = 6371000

    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance

def time_to_travel(lat1, lon1, lat2, lon2, walking_speed_kmh=5):
    distance = haversine(lat1, lon1, lat2, lon2)

    walking_speed_ms = (walking_speed_kmh * 1000) / 3600 # из км/ч в м/с

    time_seconds = distance / walking_speed_ms # время в пути в секундах

    time_minutes = time_seconds / 60 # время в пути в минутах

    return round(distance, 2), round(time_minutes, 2)

In [171]:
office_coordinates = [
    (59.940289, 30.369587),  # пер. Виленский, дом 14 литера А
    (59.938563, 30.384499),  # Дегтярный пер., д. 11Б
    (59.901766, 30.323836)   # Киевская ул., 5 корпус 4
]

for _, restaurant in df_processed.iterrows():
    restaurant_point = (restaurant['point_lat'], restaurant['point_lon'])

    for office_idx, office in enumerate(office_coordinates):
        distance, duration = time_to_travel(office[0], office[1], restaurant['point_lat'], restaurant['point_lon'])

        df_processed.loc[restaurant.name, f'office_{office_idx+1}_dist'] = distance
        df_processed.loc[restaurant.name, f'office_{office_idx+1}_time'] = duration

In [172]:
df_processed.to_csv('data/data_restaurants_processed2.csv', index=False)

In [173]:
df_processed.head()

,name,reviews_general_rating,id,name_ex_extension,name_ex_primary,address_building_id,address_components_0_number,address_components_0_street,address_name,adm_div_1_name,...,Sat_from,Sat_to,Sun_from,Sun_to,office_1_dist,office_1_time,office_2_dist,office_2_time,office_3_dist,office_3_time
0,"Harbor, ресторан",4.6,70000001062703679,ресторан,Harbor,5348660212708350,3,9-я Советская улица,"9-я Советская улица, 3",Санкт-Петербург,...,07:30:00,23:00:00,07:30:00,23:00:00,202.01,2.42,768.95,9.23,4853.62,58.24
1,"Kroo cafe, французский ресторан",4.3,5348553838944141,французский ресторан,Kroo cafe,5348660212708114,27,Суворовский проспект,"Суворовский проспект, 27",Санкт-Петербург,...,09:30:00,00:00:00,10:00:00,23:00:00,397.89,4.77,584.16,7.01,4870.39,58.44
2,"Osteria Betulla, остерия",4.3,70000001050842811,остерия,Osteria Betulla,5348660212708348,29,Греческий проспект,"Греческий проспект, 29",Санкт-Петербург,...,12:00:00,23:30:00,12:00:00,23:30:00,184.60,2.22,792.92,9.52,4850.67,58.21
3,"Feromon Group, лаундж-бар",4.7,70000001066331086,лаундж-бар,Feromon Group,5348660212794938,4а,8-я Советская улица,"8-я Советская улица, 4а",Санкт-Петербург,...,10:00:00,06:00:00,10:00:00,06:00:00,250.46,3.01,775.66,9.31,4807.53,57.69
4,"Степнов, ресторан",4.6,5348552840330064,ресторан,Степнов,5348660212708111,25,Суворовский проспект,"Суворовский проспект, 25",Санкт-Петербург,...,12:00:00,23:00:00,12:00:00,23:00:00,416.80,5.00,617.63,7.41,4821.52,57.86


#### Добавляем дополнительные колонки для основных видов кухни и основных ограничений

In [21]:
cuisines = {
    "белорусская кухня": ["Европейская кухня", "Русская кухня"],
    "средиземноморская кухня": ["Европейская кухня"],
    "сербская кухня": ["Европейская кухня"],
    "авторская кухня": [],
    "кавказская кухня": ["Европейская кухня", "Грузинская кухня"],
    "арабская кухня": ["Паназиатская кухня"],
    "гавайская кухня": ["Американская кухня"],
    "карельская кухня": ["Русская кухня", "Европейская кухня"],
    "еврейская кухня": ["Европейская кухня", "Русская кухня"],
    "европейская кухня": ["Европейская кухня"],
    "паназиатская кухня": ["Паназиатская кухня"],
    "вьетнамская кухня": ["Паназиатская кухня"],
    "итальянская кухня": ["Европейская кухня"],
    "испанская кухня": ["Европейская кухня"],
    "турецкая кухня": ["Европейская кухня", "Паназиатская кухня"],
    "азиатская кухня": ["Паназиатская кухня"],
    "армянская кухня": ["Европейская кухня", "Грузинская кухня"],
    "индийская кухня": ["Паназиатская кухня"],
    "американская кухня": ["Американская кухня"],
    "азербайджанская кухня": ["Европейская кухня", "Грузинская кухня"],
    "чешская кухня": ["Европейская кухня"],
    "грузинская кухня": ["Европейская кухня", "Грузинская кухня"],
    "халяльная кухня": ["Паназиатская кухня"],
    "французская кухня": ["Европейская кухня"],
    "восточная кухня": ["Паназиатская кухня"],
    "израильская кухня": ["Европейская кухня"],
    "рыбная кухня": ["Европейская кухня", "Русская кухня"],
    "мексиканская кухня": ["Американская кухня"],
    "веганская кухня": [],
    "русская кухня": ["Русская кухня", "Европейская кухня"],
    "тайская кухня": ["Паназиатская кухня"],
    "корейская кухня": ["Паназиатская кухня"],
    "вегетарианская кухня": [],
    "дагестанская кухня": ["Грузинская кухня"],
    "домашняя кухня": [],
    "греческая кухня": ["Европейская кухня"],
    "немецкая кухня": ["Европейская кухня"],
    "украинская кухня": ["Европейская кухня", "Русская кухня"],
    "китайская кухня": ["Паназиатская кухня"],
    "ливанская кухня": ["Паназиатская кухня"],
    "узбекская кухня": ["Паназиатская кухня"],
    "японская кухня": ["Паназиатская кухня"]
}

main_cuisines = ['Европейская кухня', 'Паназиатская кухня', 'Русская кухня', 'Американская кухня', 'Грузинская кухня']

In [22]:
df_processed = pd.read_csv('data/data_restaurants_processed2.csv');

In [23]:
def check_cuisines(cuisines_str):
    if pd.isna(cuisines_str):
        return {cuisine: False for cuisine in main_cuisines}
    
    result = {cuisine: False for cuisine in main_cuisines}  
    cuisines_list = str(cuisines_str).split('; ')  
    
    for cuisine in cuisines_list:
        for main_cuisine in main_cuisines:
            if main_cuisine in cuisines[cuisine.lower()]:  
                result[main_cuisine] = True
    return result

df_processed[main_cuisines] = df_processed['Cuisine'].apply(check_cuisines).apply(pd.Series)

In [24]:
def check_postnoe_menu(cuisine_str, dishes_str, check_str):
    if pd.notna(cuisine_str) and check_str in str(cuisine_str).lower():
        return True
    if pd.notna(dishes_str) and check_str in str(dishes_str).lower():
        return True
    return False

df_processed['Постное меню'] = df_processed.apply(lambda row: check_postnoe_menu(row['Cuisine'], row['Dishes'], "постное меню"), axis=1)
df_processed['Вегетарианское меню'] = df_processed.apply(lambda row: check_postnoe_menu(row['Cuisine'], row['Dishes'], ("веган" or "вегет")), axis=1)

In [32]:
df_processed[main_cuisines + ['Постное меню', 'Вегетарианское меню', 'Cuisine']].head()

,Европейская кухня,Паназиатская кухня,Русская кухня,Американская кухня,Грузинская кухня,Постное меню,Вегетарианское меню,Cuisine
0,True,False,False,True,False,False,False,Американская кухня; Европейская кухня
1,True,False,False,False,False,True,False,Европейская кухня; Французская кухня
2,True,False,False,False,False,False,False,Авторская кухня; Итальянская кухня
3,True,True,False,False,False,False,False,Европейская кухня; Паназиатская кухня; Итальян...
4,True,False,True,False,False,True,False,Европейская кухня; Русская кухня; Авторская кухня


In [31]:
df_processed.to_csv('data/data_restaurants_processed3.csv', index=False)